## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-28-03-54-46 +0000,nyt,Jimmy Lai’s Freedom May Now Hinge on Beijing a...,https://www.nytimes.com/2025/08/27/world/asia/...
1,2025-08-28-03-46-51 +0000,nypost,4 New Mexico deputies fired in wake of viral v...,https://nypost.com/2025/08/27/us-news/4-new-me...
2,2025-08-28-03-33-35 +0000,startribune,"In homer-happy game, double off wall sinks Twi...",https://www.startribune.com/minnesota-twins-to...
3,2025-08-28-03-32-00 +0000,wsj,President Trump on Wednesday fired Robert Prim...,https://www.wsj.com/politics/policy/trump-fire...
4,2025-08-28-03-26-28 +0000,bbc,Kim Jong Un to join Putin and other leaders at...,https://www.bbc.com/news/articles/c17njr88ge4o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2291/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
6,trump,55
100,school,15
156,shooting,15
155,minneapolis,13
7,new,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
35,2025-08-28-01-00-00 +0000,wsj,"In his first term, administration officials re...",https://www.wsj.com/politics/policy/in-trumps-...,148
192,2025-08-27-16-15-57 +0000,nypost,Cracker Barrel changed logo after call with Wh...,https://nypost.com/2025/08/27/business/cracker...,104
228,2025-08-27-14-30-00 +0000,wsj,A Beijing-linked espionage campaign that hit U...,https://www.wsj.com/politics/national-security...,99
234,2025-08-27-13-51-00 +0000,nypost,Trump administration announces it’s going to r...,https://nypost.com/2025/08/27/us-news/trump-ad...,96
3,2025-08-28-03-32-00 +0000,wsj,President Trump on Wednesday fired Robert Prim...,https://www.wsj.com/politics/policy/trump-fire...,93


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
35,148,2025-08-28-01-00-00 +0000,wsj,"In his first term, administration officials re...",https://www.wsj.com/politics/policy/in-trumps-...
183,70,2025-08-27-16-45-28 +0000,nyt,The Catholic school where the shooting took pl...,https://www.nytimes.com/live/2025/08/27/us/min...
111,56,2025-08-27-21-29-33 +0000,nypost,Zohran Mamdani would easily win NYC mayor race...,https://nypost.com/2025/08/27/us-news/zohran-m...
228,44,2025-08-27-14-30-00 +0000,wsj,A Beijing-linked espionage campaign that hit U...,https://www.wsj.com/politics/national-security...
125,42,2025-08-27-21-01-42 +0000,nypost,Angry Cracker Barrel workers dish on skimpy pa...,https://nypost.com/2025/08/27/business/angry-c...
264,40,2025-08-27-11-15-50 +0000,cbc,Denmark summons top U.S. diplomat after report...,https://www.cbc.ca/news/world/denmark-us-green...
283,40,2025-08-27-10-00-00 +0000,nypost,Grassley insists to ‘Pod Force One’ that Biden...,https://nypost.com/2025/08/27/us-news/grassley...
6,38,2025-08-28-03-09-00 +0000,wsj,"Susan Monarez, the director of the Centers for...",https://www.wsj.com/politics/policy/cdc-direct...
97,38,2025-08-27-22-00-06 +0000,nypost,NYC jewelers insist Travis Kelce could’ve gott...,https://nypost.com/2025/08/27/us-news/nyc-jewe...
165,37,2025-08-27-17-56-38 +0000,latimes,"Cal Fire captain fatally shot his girlfriend, ...",https://www.latimes.com/california/story/2025-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
